# CARLA Python API - Lab4

Lab goals:
- use camera/GNSS/LIDAR sensors for engaging driving tasks;
- structure scripts with helpers + clean cleanup.

## CARLA documentation links
- Main docs: https://carla.readthedocs.io/en/latest/
- Sensors: https://carla.readthedocs.io/en/latest/ref_sensors/
- Python API: https://carla.readthedocs.io/en/latest/python_api/
- Debug helper: https://carla.readthedocs.io/en/latest/python_api/#carladebughelper
- Vehicle lights: https://carla.readthedocs.io/en/latest/python_api/#carlavehiclelightstate


In [ ]:
import carla, time, random
import numpy as np


In [ ]:
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
spectator = world.get_spectator()


## 1. Shared helper functions

Docs used in this section:
- Actors and spawning: https://carla.readthedocs.io/en/latest/core_actors/
- Camera sensor: https://carla.readthedocs.io/en/latest/ref_sensors/#rgb-camera


In [ ]:
def move_spectator_to(transform, spectator, distance=7.0, z=3.0, pitch=-15.0):
    back = transform.location - transform.get_forward_vector() * distance
    loc = carla.Location(back.x, back.y, back.z + z)
    rot = carla.Rotation(pitch=pitch, yaw=transform.rotation.yaw, roll=0.0)
    spectator.set_transform(carla.Transform(loc, rot))


def spawn_vehicle(world, spawn_index=0, vehicle_filter="vehicle.tesla.model3", autopilot=False):
    points = world.get_map().get_spawn_points()
    if not points:
        raise RuntimeError("No spawn points found")
    bps = world.get_blueprint_library().filter(vehicle_filter)
    if not bps:
        bps = world.get_blueprint_library().filter("vehicle.*")
    for k in range(len(points)):
        actor = world.try_spawn_actor(random.choice(bps), points[(spawn_index + k) % len(points)])
        if actor is not None:
            actor.set_autopilot(autopilot)
            return actor
    raise RuntimeError("Could not spawn vehicle")


def spawn_camera(world, attach_to, transform, width=640, height=360, fov=95, tick=0.05):
    bp = world.get_blueprint_library().find("sensor.camera.rgb")
    bp.set_attribute("image_size_x", str(width))
    bp.set_attribute("image_size_y", str(height))
    bp.set_attribute("fov", str(fov))
    bp.set_attribute("sensor_tick", str(tick))
    return world.spawn_actor(bp, transform, attach_to=attach_to)


def image_to_bgr(image):
    arr = np.frombuffer(image.raw_data, dtype=np.uint8)
    arr = np.reshape(arr, (image.height, image.width, 4))
    return arr[:, :, :3][:, :, ::-1].copy()


def safe_destroy(actors):
    for a in actors:
        if a is not None:
            try:
                a.destroy()
            except RuntimeError:
                pass


## 2. Exercises (in class)

Reuse the helper functions above.
Each exercise includes a guided template with `STEP` + `TODO`.


### Exercise 1: Streamer cockpit (4 cameras)

Build 4 synchronized views:
- front windshield
- left mirror
- right mirror
- rear view

Docs:
- RGB camera: https://carla.readthedocs.io/en/latest/ref_sensors/#rgb-camera
- Transform API: https://carla.readthedocs.io/en/latest/python_api/#carlatransform


In [ ]:
# TEMPLATE - Exercise 1
vehicle = None
cameras = []
frames = {}

specs = {
    "front": carla.Transform(carla.Location(x=1.8, z=1.4), carla.Rotation(pitch=-6.0)),
    "left": carla.Transform(carla.Location(x=0.1, y=-0.75, z=1.3), carla.Rotation(yaw=-145.0)),
    "right": carla.Transform(carla.Location(x=0.1, y=0.75, z=1.3), carla.Rotation(yaw=145.0)),
    "rear": carla.Transform(carla.Location(x=-2.1, z=1.3), carla.Rotation(yaw=180.0)),
}

try:
    # STEP 1: spawn ego vehicle with autopilot
    # TODO

    # STEP 2: spawn one camera per view and store frames
    # TODO

    # STEP 3: callback factory per camera name
    # TODO

    # STEP 4: show all windows, quit with 'q'
    # TODO
    pass
finally:
    # STEP 5: stop sensors + destroy actors + close windows
    # TODO
    pass


### Exercise 2: Auto headlights from image brightness

Implement automatic low beams using RGB brightness with hysteresis.

Docs:
- VehicleLightState: https://carla.readthedocs.io/en/latest/python_api/#carlavehiclelightstate
- RGB camera: https://carla.readthedocs.io/en/latest/ref_sensors/#rgb-camera


In [ ]:
# TEMPLATE - Exercise 2
vehicle = None
camera = None
state = {"brightness": 255.0, "lights_on": False}
turn_on = 70.0
turn_off = 95.0

try:
    # STEP 1: spawn vehicle + front camera
    # TODO

    def on_image(image):
        # STEP 2: compute mean grayscale brightness
        # hint: gray = cv2.cvtColor(image_to_bgr(image), cv2.COLOR_BGR2GRAY)
        # TODO
        pass

    # TODO: camera.listen(on_image)

    while True:
        # STEP 3: hysteresis logic for lights
        # TODO
        world.tick()
        time.sleep(0.03)
except KeyboardInterrupt:
    pass
finally:
    # STEP 4: cleanup
    # TODO
    pass


### Exercise 3: GPS Pacman portals

Make the car move continuously and teleport at boundary crossing.

Docs:
- GNSS sensor: https://carla.readthedocs.io/en/latest/ref_sensors/#gnss-sensor
- set_transform: https://carla.readthedocs.io/en/latest/python_api/


In [ ]:
# TEMPLATE - Exercise 3
vehicle = None
gnss = None
state = {"lat": None, "lon": None}
X_MIN = -110.0
X_MAX = 110.0

try:
    # STEP 1: spawn vehicle + attach GNSS sensor
    # TODO

    def on_gnss(event):
        # STEP 2: update GNSS values
        # TODO
        pass

    # TODO: gnss.listen(on_gnss)

    while True:
        # STEP 3: keep throttle > 0
        # STEP 4: if x > X_MAX -> teleport to X_MIN and draw "PORTAL!"
        # STEP 5: print GNSS every second
        # TODO
        world.tick()
        time.sleep(0.03)
except KeyboardInterrupt:
    pass
finally:
    # STEP 6: cleanup
    # TODO
    pass


### Optional Exercise 4: LIDAR safety bubble

Detect nearest obstacle in front sector and apply progressive braking.

Docs:
- LIDAR sensor: https://carla.readthedocs.io/en/latest/ref_sensors/#lidar-sensor
- VehicleControl: https://carla.readthedocs.io/en/latest/python_api/#carlavehiclecontrol


In [ ]:
# TEMPLATE - Optional Exercise 4
vehicle = None
target = None
lidar = None
state = {"d_front": None}

try:
    # STEP 1: spawn ego + target ahead
    # TODO

    # STEP 2: attach lidar to ego
    # TODO

    def on_lidar(measurement):
        # STEP 3: parse raw_data (Nx4 float32), keep front points, compute min distance
        # TODO
        pass

    # TODO: lidar.listen(on_lidar)

    while True:
        # STEP 4: throttle/brake based on distance thresholds
        # TODO
        world.tick()
        time.sleep(0.03)
except KeyboardInterrupt:
    pass
finally:
    # STEP 5: cleanup
    # TODO
    pass
